In [4]:
from kubernetes import config, dynamic
from kubernetes import client as k8s_client
from kubernetes.dynamic.exceptions import ResourceNotFoundError
from kubernetes.client import api_client
import time

In [60]:
# client = dynamic.DynamicClient(
#         api_client.ApiClient(configuration=config.load_kube_config())
#     )

#     # fetching the custom resource definition (CRD) api
# crd_api = client.resources.get(
#     api_version="apiextensions.k8s.io/v1", kind="CustomResourceDefinition"
# )

# namespace_api = client.resources.get(api_version="v1", kind="Namespace")

# print(namespace_api)

In [19]:
# !pip install kubernetes

In [5]:
def list_ingressroute_for_all_namespaces(group, version, plural):
    custom_object_api = k8s_client.CustomObjectsApi()

    list_of_ingress_routes = custom_object_api.list_cluster_custom_object(
        group, version, plural
    )
    print(list_of_ingress_routes)
    # print(
    #     "%s\t\t\t%s\t\t\t%s\t\t%s\t\t\t\t%s"
    #     % ("NAME", "NAMESPACE", "FQDN", "TLS", "STRATEGY")
    # )
    # for item in list_of_ingress_routes["items"]:
    #     print(
    #         "%s\t%s\t\t%s\t%s\t%s"
    #         % (
    #             item["metadata"]["name"],
    #             item["metadata"]["namespace"],
    #             item["spec"]["virtualhost"]["fqdn"],
    #             item["spec"]["virtualhost"]["tls"],
    #             item["spec"]["strategy"]
    #         )
    #     )

In [7]:
# list_ingressroute_for_all_namespaces(
#         group="blackadder.io", version="v1betav1", plural="chaosagents"
#     )

In [15]:
def main():
    # Creating a dynamic client
    client = dynamic.DynamicClient(
        api_client.ApiClient(configuration=config.load_kube_config())
    )

    # fetching the node api
    api = client.resources.get(api_version="v1alpha1", kind="ChaosAgent")
    # len(api)

    # Listing cluster nodes

    print("%s\t\t%s\t\t%s\t\t%s\t\t%s\t\t%s\t\t%s" % ("NAME", "TANTRUM_MODE", "POD_TOLERANCE","cancerMode","ipsumMode","eagerness","pauseDuration"))
    for item in api.get().items:
        node = api.get(name=item.metadata.name)
        
        print(
            "%s\t%s\t\t\t\t%s\t\t%s\t\t\t%s\t\t\t%s\t\t\t%s\n"
            % (
                node.metadata.name,
                node.spec.tantrumMode,
                node.spec.podTolerance,
                node.spec.cancerMode,
                node.spec.ipsumMode,
                node.spec.eagerness,
                node.spec.pauseDuration,
                # node.status.conditions[3]["type"],
                # node.status.nodeInfo.kubeProxyVersion,
            )
        )
    print("Number of Pod running : ", len(api.get().items))

if __name__ == "__main__":
    main()

NAME		TANTRUM_MODE		POD_TOLERANCE		cancerMode		ipsumMode		eagerness		pauseDuration
princeedmund1	True				10		None			None			None			None

Number of Pod running :  1


In [17]:
!kubectl get chaosagents.blackadder.io

NAME            TANTRUM   TOLERANCE   CANCER   IPSUM   EAGERNESS   PAUSE   EXCLUDED
princeedmund1   true      10          true             20          30      ["kube-system"]


In [8]:
# !pip install pykube

In [6]:
import random
import sys
import time

import lorem
import munch
import pykube
import requests


from pykube import Pod, Deployment, ConfigMap


def list_objects(self, k8s_obj, exclude_namespaces):

    exclude_namespaces = ",".join("metadata.namespace!=" + ns
                                  for ns in exclude_namespaces)
    return list(
            k8s_obj.objects(api).filter(namespace=pykube.all,
                                        field_selector=exclude_namespaces

                                        ))


config = pykube.KubeConfig.from_env()


pykube.HTTPClient.list_objects = list_objects


api = pykube.HTTPClient(config)

ChaosAgent = pykube.object_factory(api, "blackadder.io/v1beta1", "ChaosAgent")

# retrieves our agent configuraton from the kube-api-server
agent = list(ChaosAgent.objects(api, namespace=pykube.all))[0]

agent.config = munch.munchify(agent.obj["spec"])

exclude_namespaces = agent.config.excludedNamespaces


def randomly_kill_pods(pods, tolerance, eagerness):
    if len(pods) < tolerance:
        return

    for p in pods:
        if random.randint(0, 100) < eagerness:
            p.delete()
            print(f"Deleted {p.namespace}/{p.name}",)


def randomly_scale_deployments(deployments, eagerness):
    for d in deployments:
        if random.randint(0, 100) < eagerness:
            while True:
                try:
                    if d.replicas < 128:
                        d.replicas = min(d.replicas * 2, 128)
                    d.update()
                    print(f"scaled {d.namespace}/{d.name} to {d.replicas}",)
                    break
                except (requests.exceptions.HTTPError, pykube.exceptions.HTTPError):
                    print(
                        f"error scaling {d.namespace}/{d.name} to {d.replicas}",)
                    d.reload()
                    continue


def randomly_write_configmaps(configmaps, eagerness):
    for cm in configmaps:
        print(f"Checking {cm.namespace}/{cm.name}")
        if cm.obj.get("immutable"):
            continue

        if random.randint(0, 100) < eagerness:
            for k, v in cm.obj["data"].items():
                cm.obj["data"][k] = lorem.paragraph()

            print(f"Lorem Impsum in {cm.namespace}/{cm.name}",)


def main():
    while True:
        pods = api.list_objects(Pod, exclude_namespaces)
        deployments = api.list_objects(Deployment, exclude_namespaces)
        configmaps = api.list_objects(ConfigMap, exclude_namespaces)

        if agent.config.tantrumMode:
            randomly_kill_pods(pods,
                               agent.config.podTolerance,
                               agent.config.eagerness)

        if agent.config.cancerMode:
            randomly_scale_deployments(deployments,
                                       agent.config.eagerness)

        if agent.config.ipsumMode:
            randomly_write_configmaps(configmaps,
                                      agent.config.eagerness)

        time.sleep(agent.config.pauseDuration)


if __name__ == "__main__":
    print("This is the blackadder version 0.1.1")
    print("Ready to start a havoc in your cluster")
    main()

This is the blackadder version 0.1.1
Ready to start a havoc in your cluster
Checking default/kube-root-ca.crt
Checking kube-node-lease/kube-root-ca.crt
Checking kube-public/cluster-info
Checking kube-public/kube-root-ca.crt
Checking default/kube-root-ca.crt
Checking kube-node-lease/kube-root-ca.crt
Checking kube-public/cluster-info
Checking kube-public/kube-root-ca.crt
Checking default/kube-root-ca.crt
Checking kube-node-lease/kube-root-ca.crt
Checking kube-public/cluster-info
Lorem Impsum in kube-public/cluster-info
Checking kube-public/kube-root-ca.crt
Checking default/kube-root-ca.crt
Checking kube-node-lease/kube-root-ca.crt
Checking kube-public/cluster-info
Lorem Impsum in kube-public/cluster-info
Checking kube-public/kube-root-ca.crt
Checking default/kube-root-ca.crt
Checking kube-node-lease/kube-root-ca.crt
Checking kube-public/cluster-info
Checking kube-public/kube-root-ca.crt
Lorem Impsum in kube-public/kube-root-ca.crt
Checking default/kube-root-ca.crt
Checking kube-node-leas

In [5]:
pip install pykube-ng

     ---------------------------------------- 0.0/48.9 kB ? eta -:--:--
     ------------------------------- ------ 41.0/48.9 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 48.9/48.9 kB 822.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/143.8 kB ? eta -:--:--
   ---------------------------------- ----- 122.9/143.8 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 143.8/143.8 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.8
    Uninstalling urllib3-1.26.8:
      Successfully uninstalled urllib3-1.26.8
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.


In [20]:
!pip install pykube-ng


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install pykube-ng
pip uninstall pykube-ng
pip install pykube
pip uninstall pykube

In [6]:
# Config map
# Creating a dynamic client
client = dynamic.DynamicClient(
    api_client.ApiClient(configuration=config.load_kube_config())
)

# fetching the node api
api = client.resources.get(api_version="v1alpha1", kind="ChaosAgent")
for item in api.get().items:
    print(item.metadata)

{'annotations': {'kubectl.kubernetes.io/last-applied-configuration': '{"apiVersion":"blackadder.io/v1alpha1","kind":"ChaosAgent","metadata":{"annotations":{},"name":"princeedmund1"},"spec":{"podTolerance":10,"tantrumMode":true}}\n'},
 'creationTimestamp': '2023-11-24T06:01:37Z',
 'generation': 1,
 'managedFields': [{'apiVersion': 'blackadder.io/v1alpha1',
 'fieldsType': 'FieldsV1',
 'fieldsV1': {'f:metadata': {'f:annotations': {'.': {}, 'f:kubectl.kubernetes.io/last-applied-configuration': {}}},
 'f:spec': {'.': {}, 'f:podTolerance': {}, 'f:tantrumMode': {}}},
 'manager': 'kubectl-client-side-apply',
 'operation': 'Update',
 'time': '2023-11-24T06:01:37Z'}],
 'name': 'princeedmund1',
 'resourceVersion': '13470',
 'uid': '63559b01-64e3-46f9-843e-75f0e4b5ec11'}


In [81]:
# get the cluster scoped resource
from kubernetes import client, config
config.load_kube_config()

api1 = client.CustomObjectsApi()
resource = api1.get_cluster_custom_object(
    group="blackadder.io",
    version="v1alpha1",
    name="chaosagents.blackadder.io",
    plural="chaosagents",
)
# print(resource)
# group="blackadder.io", version="v1alpha1", plural="chaosagents"
# print("%s\t\t%s" % ("NAME", "CRON-SPEC"))
# print(f"{resource['metadata']['name']}\t{resource['spec']['cronSpec']}\n")

ApiException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Audit-Id': '3a969ce9-5ede-46e8-af61-de4a1ef12da3', 'Cache-Control': 'no-cache, private', 'Content-Type': 'application/json', 'X-Kubernetes-Pf-Flowschema-Uid': '2e3c464e-36e2-4c7e-ab3b-65343e80622e', 'X-Kubernetes-Pf-Prioritylevel-Uid': '049503df-5f6a-4ddf-9c8c-b6ba625f396c', 'Date': 'Fri, 24 Nov 2023 11:44:43 GMT', 'Content-Length': '270'})
HTTP response body: {"kind":"Status","apiVersion":"v1","metadata":{},"status":"Failure","message":"chaosagents.blackadder.io \"chaosagents.blackadder.io\" not found","reason":"NotFound","details":{"name":"chaosagents.blackadder.io","group":"blackadder.io","kind":"chaosagents"},"code":404}



In [10]:
!kubectl get nodes

NAME            STATUS   ROLES           AGE     VERSION
entwickler.de   Ready    control-plane   6h41m   v1.27.4


In [10]:
!kubectl apply -f blackadder-v1alpha1.yml

customresourcedefinition.apiextensions.k8s.io/chaosagents.blackadder.io configured


In [11]:
!kubectl apply -f edmund.yml

chaosagent.blackadder.io/princeedmund1 configured


In [6]:
!kubectl get namespace

NAME              STATUS   AGE
default           Active   26h
kube-node-lease   Active   26h
kube-public       Active   26h
kube-system       Active   26h


In [12]:
!kubectl get all -n default

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   6h41m


In [17]:
!kubectl get chaosagents.blackadder.io

error: the server doesn't have a resource type "chaosagents"


In [19]:
client = connect_to_kubernetes()

# retrieves our agent configuration from the kube-api-server
chaos_agent = client.get_chaos_agent()

while True:
    pods = client.list_pods(exclude_namespaces)
    deployments = client.list_deployments(exclude_namespaces)
    namespaces = client.list_configmaps(exclude_namespaces)
    if chaos_agent.tantrum:
        randomly_kill_pods(pods, chaos_agent.tolerance, chaos_agent.eagerness)

    if chaos_agent.cancer:
        randomly_scale_deployments(deployments, chaos_agent.eagerness)
    if chaos_agent.ipsum:
        randomly_write_configmaps(configmaps, chaos_agent.eagerness)

    time.sleep(chaos_agent.pause)

NameError: name 'connect_to_kubernetes' is not defined

In [48]:
import random
import sys
import time

import lorem
import munch
import pykube
import requests


from pykube import Pod, Deployment, ConfigMap


def list_objects(self, k8s_obj, exclude_namespaces):

    exclude_namespaces = ",".join("metadata.namespace!=" + ns
                                  for ns in exclude_namespaces)
    return list(
            k8s_obj.objects(api).filter(namespace=pykube.all,
                                        field_selector=exclude_namespaces

                                        ))


# config = pykube.KubeConfig.from_env('~/.kube/config')
config=pykube.KubeConfig.from_file('~/.kube/config')


pykube.HTTPClient.list_objects = list_objects


api = pykube.HTTPClient(config)
print(api)

ChaosAgent = pykube.object_factory(api, "blackadder.io/v1alpha1", "ChaosAgent")

# retrieves our agent configuraton from the kube-api-server
agent = list(ChaosAgent.objects(api, namespace=pykube.all))[0]

agent.config = munch.munchify(agent.obj["spec"])

exclude_namespaces = agent.config.excludedNamespaces


def randomly_kill_pods(pods, tolerance, eagerness):
    if len(pods) < tolerance:
        return

    for p in pods:
        if random.randint(0, 100) < eagerness:
            p.delete()
            print(f"Deleted {p.namespace}/{p.name}",)


def randomly_scale_deployments(deployments, eagerness):
    for d in deployments:
        if random.randint(0, 100) < eagerness:
            while True:
                try:
                    if d.replicas < 128:
                        d.replicas = min(d.replicas * 2, 128)
                    d.update()
                    print(f"scaled {d.namespace}/{d.name} to {d.replicas}",)
                    break
                except (requests.exceptions.HTTPError, pykube.exceptions.HTTPError):
                    print(
                        f"error scaling {d.namespace}/{d.name} to {d.replicas}",)
                    d.reload()
                    continue


def randomly_write_configmaps(configmaps, eagerness):
    for cm in configmaps:
        print(f"Checking {cm.namespace}/{cm.name}")
        if cm.obj.get("immutable"):
            continue

        if random.randint(0, 100) < eagerness:
            for k, v in cm.obj["data"].items():
                cm.obj["data"][k] = lorem.paragraph()

            print(f"Lorem Impsum in {cm.namespace}/{cm.name}",)


def main():
    while True:
        pods = api.list_objects(Pod, exclude_namespaces)
        deployments = api.list_objects(Deployment, exclude_namespaces)
        configmaps = api.list_objects(ConfigMap, exclude_namespaces)

        if agent.config.tantrumMode:
            randomly_kill_pods(pods,
                               agent.config.podTolerance,
                               agent.config.eagerness)

        if agent.config.cancerMode:
            randomly_scale_deployments(deployments,
                                       agent.config.eagerness)

        if agent.config.ipsumMode:
            randomly_write_configmaps(configmaps,
                                      agent.config.eagerness)

        time.sleep(agent.config.pauseDuration)


if __name__ == "__main__":
    print("This is the blackadder version 0.1.1")
    print("Ready to start a havoc in your cluster")
    main()

AttributeError: module 'pykube' has no attribute 'object_factory'

In [22]:
pip install lorem 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
pip install munch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
pip install pykube

  Using cached pykube-0.15.0-py2.py3-none-any.whl (23 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install pykube-ng

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
!pip uninstall pykube

^C
